In [1]:
import torch
import numpy as np
import os

c:\Users\itsjo\anaconda3\envs\test_env\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\itsjo\anaconda3\envs\test_env\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\itsjo\anaconda3\envs\test_env\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
path_to_labels = '../Labels/Test_Subject_3'
os.listdir(path_to_labels)

['By_1', 'By_2']

In [3]:
# cohen's kappa
# preds -> shape (N, ...) or float tensor of shape (N, C, ...)
# target -> shape (N, ...)

In [4]:
from torchmetrics.classification import MulticlassCohenKappa

c:\Users\itsjo\anaconda3\envs\test_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
%cd ../../

c:\Users\itsjo\Documents\repos\assembly_glovebox_dataset


In [6]:
from training.dataloaders.datamodule import AssemblyDataModule

In [7]:
# TODO: make it so you can just say the participants name and attain everything else
dm = AssemblyDataModule(
        fit_query= None,
        test_query = {
            # queries can be the same as above
            'participants': ['Test_Subject_3'], 
            'distribution':['ood', 'id'],
            'task': ['J', 'TB'], 
            'view': ['Top_View', 'Side_View'],
        }
    )

Initialized None and {'participants': ['Test_Subject_3'], 'distribution': ['ood', 'id'], 'task': ['J', 'TB'], 'view': ['Top_View', 'Side_View']}


In [8]:
dm.setup(stage="test")

In [9]:
dataloaders = dm.predict_dataloader()

In [10]:
dataloaders

In [11]:
def get_cohen(y1, y2):
    cohenkappa = MulticlassCohenKappa(num_classes=3).to("cuda")
    return cohenkappa(y1, y2)

In [ ]:
cohens = []
for batch in dataloaders:
    x, y = batch
    x = x.to("cuda")
    y1 = y[0].to("cuda")
    y2 = y[1].to("cuda")
    
    c = get_cohen(y1, y2)
    cohens.append(c)
    # had to change num_workers=0 to satisfy windows

In [33]:
mean_cohen = torch.mean(torch.stack(cohens))
mean_cohen.item()

0.8920516967773438